In [ ]:
%pylab inline
import numpy as np
import pandas as pd

In [ ]:
plt.style.use('bmh')

In [ ]:
import seaborn as sns
sns.set()

# Loading data

In [ ]:
d = pd.read_csv('data/accidents_2005_to_2007.csv.zip')

In [ ]:
d.head()

In [ ]:
d.info(memory_usage="deep")

In [ ]:
d.Date.head()

In [ ]:
d.Time.head()

In [ ]:
d.loc[:, 'dt'] = d.Date.str.cat(d.Time, sep=' ', na_rep='00:00')

In [ ]:
d.loc[:, 'date_time'] = pd.to_datetime(d.dt, dayfirst=True)

In [ ]:
d.info()

In [ ]:
d.columns

In [ ]:
COLS = [u'Accident_Index', 'Longitude', 'Latitude', 'Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties', 'Weather_Conditions',
       'Day_of_Week', u'Weather_Conditions', u'Road_Surface_Conditions', u'Special_Conditions_at_Site',
        u'Urban_or_Rural_Area', u'Carriageway_Hazards', 'date_time']

In [ ]:
d.drop([c for c in d.columns if c not in COLS], axis=1, inplace=True)

In [ ]:
d.set_index('date_time', inplace=True)

In [ ]:
d.index

# Resampling time series

In [ ]:
daily = d.resample('D').size()
daily

In [ ]:
daily.index

In [ ]:
daily.index.is_monotonic, daily.index.is_unique

In [ ]:
plt.figure(figsize=(8,3), dpi=150)
daily.plot(ax=plt.gca(), linewidth=0.5)
(d.resample('W').size()/7.).plot(ax=plt.gca(), linewidth=1, color='firebrick')
plt.ylabel('accidents')
plt.xlabel('month');

In [ ]:
plt.figure(figsize=(12,5))

d.resample('D').Number_of_Vehicles.mean().plot(ax=plt.gca())
d.resample('W').Number_of_Vehicles.mean().plot(ax=plt.gca(), color='firebrick')

plt.ylabel('vehicles involved')
plt.xlabel('month');

In [ ]:
plt.figure(figsize=(12,6))

(d.groupby([pd.Grouper(freq='1M'), 'Urban_or_Rural_Area'])
 .size()
 .unstack()
 .plot(alpha=0.6, linewidth=2, ax=plt.gca(), kind='bar', stacked=True));

In [ ]:
d.Weather_Conditions.value_counts()

In [ ]:
plt.figure(figsize=(12,6))

(d.groupby([pd.Grouper(freq='1M'), 'Accident_Severity'])
 .size()
 .unstack()
 .plot(alpha=0.6, linewidth=2, ax=plt.gca(), kind='bar', stacked=True));

In [ ]:
plt.figure(figsize=(12,6))

d.groupby('Accident_Severity').resample('W').size().T.plot(alpha=0.6, linewidth=2)

plt.legend(loc=0)

In [ ]:
d.index.min()

In [ ]:
d.groupby('Accident_Severity').resample('W').size().T

In [ ]:
d.groupby('Accident_Severity').resample('W-MON').size().T

In [ ]:
d.groupby('Weather_Conditions').resample('W').size()

# GeoPandas

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon, box

In [ ]:
gd = gpd.GeoDataFrame(d[["Accident_Index", "Accident_Severity", "Number_of_Vehicles",
                         "Number_of_Casualties", "Number_of_Casualties"]],
                      geometry=gpd.points_from_xy(*d[["Longitude", "Latitude"]].values.T))

In [ ]:
?gd.plot

In [ ]:
gd.iloc[:1000].plot(figsize=(12,12), markersize=1)

In [ ]:
gd[gd.within(box(-0.2, 51.5, -0.18, 51.52))]